### Calling Keras layers on Tensorflow tensors
* In this case, we use Keras only as a syntactical shortcut to generate an op that maps some tensor(s) input to some tensor(s) output, and that's it. 
* The optimization is done via a native TensorFlow optimizer rather than a Keras optimizer. * We don't even use any Keras Model at all!

* Let's start with a simple example: MNIST digits classification. We will build a TensorFlow digits classifier using a stack of Keras Dense layers (fully-connected layers).
* We should start by creating a TensorFlow session and registering it with Keras. This means that Keras will use the session we registered to initialize all variables that it creates internally.

In [1]:
import tensorflow as tf
sess = tf.Session()
from keras.models import Sequential
from keras.layers import Dropout, Dense, LSTM, BatchNormalization
from keras.objectives import categorical_crossentropy
from keras.metrics import categorical_accuracy as accuracy

from keras import backend as K
K.set_session(sess)

Using TensorFlow backend.


In [2]:
from mnist_loader import read_data_sets
mnist_data = read_data_sets('../data/mnist', one_hot=True)

Extracting ../data/mnist/train-images-idx3-ubyte.gz
Extracting ../data/mnist/train-labels-idx1-ubyte.gz
Extracting ../data/mnist/t10k-images-idx3-ubyte.gz
Extracting ../data/mnist/t10k-labels-idx1-ubyte.gz


In [3]:
from keras.layers import Dense

# this placeholder will contain our input digits, as flat vectors
img = tf.placeholder(tf.float32, shape=(None, 784))

#### We can easily define our model in a sequential fashion, similar to what we have done in TensorFlow

In [ ]:
# Keras layers can be called on TensorFlow tensors:
x = Dense(128, activation='relu')(img)  # fully-connected layer with 128 units and ReLU activation
x = Dense(128, activation='relu')(x)
preds = Dense(10, activation='softmax')(x)  # output layer with 10 units and a softmax activation

#### Define a placeholder for the labels, and a loss function

In [ ]:
labels = tf.placeholder(tf.float32, shape=(None, 10))

from keras.objectives import categorical_crossentropy
loss = tf.reduce_mean(categorical_crossentropy(labels, preds))

#### We can train the model with a **TensorFlow** optimizer

In [ ]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

# Initialize all variables
init_op = tf.global_variables_initializer()
sess.run(init_op)

# Run training loop
with sess.as_default():
    for i in range(100):
        batch = mnist_data.train.next_batch(50)
        train_step.run(feed_dict={img: batch[0],
                                  labels: batch[1]})

#### We can now evaluate the model:

In [ ]:
from keras.metrics import categorical_accuracy as accuracy

acc_value = accuracy(labels, preds)
with sess.as_default():
    print(acc_value.eval(feed_dict={img: mnist_data.test.images,
                                    labels: mnist_data.test.labels}))

In [ ]:
sess.close()

### Different behaviors during training and testing
* Some Keras layers (e.g. Dropout, BatchNormalization) behave differently at training time and testing time. 
* You can tell whether a layer uses the "learning phase" (train/test) by printing `layer.uses_learning_phase`, a boolean: `True` if the layer has a different behavior in training mode and test mode, `False` otherwise.
* If your model includes such layers, then you need to specify the value of the learning phase as part of feed_dict, so that your model knows whether to apply dropout/etc or not.
* The Keras learning phase (a scalar TensorFlow tensor) is accessible via the Keras backend:

In [ ]:
from keras import backend as K
print(K.learning_phase())

To make use of the learning phase, simply pass the value "1" (training mode) or "0" (test mode) to feed_dict like this (for train mode):

`train_step.run(feed_dict={x: batch[0], labels: batch[1], K.learning_phase(): 1})`

For example, here we add `Dropout` layers to our previous MNIST example:

In [ ]:
import tensorflow as tf
from keras.layers import Dropout, Dense
from keras import backend as K
from keras.objectives import categorical_crossentropy

img = tf.placeholder(tf.float32, shape=(None, 784))
labels = tf.placeholder(tf.float32, shape=(None, 10))

x = Dense(128, activation='relu')(img)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
preds = Dense(10, activation='softmax')(x)

loss = tf.reduce_mean(categorical_crossentropy(labels, preds))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

sess = tf.Session()
# Initialize all variables
init_op = tf.global_variables_initializer()
sess.run(init_op)

with sess.as_default():
    for i in range(100):
        batch = mnist_data.train.next_batch(50)
        train_step.run(feed_dict={img: batch[0],
                                  labels: batch[1],
                                  K.learning_phase(): 1})

acc_value = accuracy(labels, preds)
with sess.as_default():
    print(acc_value.eval(feed_dict={img: mnist_data.test.images,
                                    labels: mnist_data.test.labels,
                                    K.learning_phase(): 0}))
sess.close()

### Compatibility with name scopes, device scopes
Keras layers and models are fully compatible with TensorFlow name scopes. For instance, consider the following code snippet.

The weights of our LSTM layer will then be named `block1/mylstm_W_i`, `block1/mylstm_U_i`, etc...

In [ ]:
from keras.layers import LSTM

x = tf.placeholder(tf.float32, shape=(None, 20, 64))
with tf.name_scope('block1'):
    y = LSTM(32, name='mylstm')(x)

Similarly, device scopes work as you would expect:

In [ ]:
with tf.device('/gpu:0'):
    x = tf.placeholder(tf.float32, shape=(None, 20, 64))
    y = LSTM(32)(x)  # all ops / variables in the LSTM layer will live on GPU:0

### Compatibility with graph scopes
* Any Keras layer or model that you define inside a TensorFlow graph scope will have all of its variables and operations created as part of the specified graph. 

* For instance, the following works as you would expect, with all of the ops / variables in the LSTM layer created as part of our graph:

In [ ]:
from keras.layers import LSTM
import tensorflow as tf

my_graph = tf.Graph()
with my_graph.as_default():
    x = tf.placeholder(tf.float32, shape=(None, 20, 64))
    y = LSTM(32)(x)

### Compatibility with variable scopes
* Variable sharing should be done via calling a same Keras layer (or model) instance multiple times, and **NOT via TensorFlow variable scopes**. 
* A TensorFlow variable scope will have no effect on a Keras layer or model. 
* For more information about weight sharing with Keras, please see the "weight sharing" section in the functional API guide: https://keras.io/getting-started/functional-api-guide/#shared-layers

To summarize quickly how weight sharing works in Keras: by reusing the same layer instance or model instance, you are sharing its weights. Here's a simple example:

In [ ]:
# instantiate a Keras layer
lstm = LSTM(32)

# instantiate two TF placeholders
x = tf.placeholder(tf.float32, shape=(None, 20, 64))
y = tf.placeholder(tf.float32, shape=(None, 20, 64))

# encode the two tensors with the *same* LSTM weights
x_encoded = lstm(x)
y_encoded = lstm(y)

### Collecting trainable weights and state updates
* Some Keras layers (stateful RNNs and BatchNormalization layers) have internal updates that need to be run as part of each training step. 
* There are stored as a list of tensor tuples, layer.updates. You should generate assign ops for those, to be run at each training step. Here's an example:

Note that if you are using a Keras model (i.e. a `Model` instance or a `Sequential` instance), `model.udpates` behaves in the same way (and collects the updates of all underlying layers in the model).

In addition, in case you need to explicitly collect a layer's trainable weights, you can do so via `layer.trainable_weights` (or `model.trainable_weights`) to get a list of TensorFlow Variable instances:

### Using Keras models with TensorFlow
#### Converting a Keras `Sequential` model for use in a TensorFlow workflow

You have found a Keras Sequential model that you want to reuse in your TensorFlow project (consider, for instance, this VGG16 image classifier with pre-trained weights: https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3). How to proceed?

* First of all, note that if your pre-trained weights include convolutions (layers Convolution2D or Convolution1D) that were trained with Theano, you need to flip the convolution kernels when loading the weights. 
* This is due Theano and TensorFlow implementing convolution in different ways (TensorFlow actually implements correlation, much like Caffe). 
* Here's a short guide on what you need to do in this case: https://github.com/fchollet/keras/wiki/Converting-convolution-kernels-from-Theano-to-TensorFlow-and-vice-versa
* Though you probably don't need to worry about this too much as fewer and fewer people are using Theano these days.

* Let's say that you are starting from the following Keras model, and that you want to modify so that it takes as input a specific TensorFlow tensor, `my_input_tensor`. This input tensor could be a data feeder op, for instance, or the output of a previous TensorFlow model.

In [ ]:
from keras.models import Sequential
from keras.layers import InputLayer

# this is our initial Keras model
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=784))
model.add(Dense(10, activation='softmax'))

You just need to use `keras.layers.InputLayer` to start building your Sequential model on top of a custom TensorFlow placeholder, then build the rest of the model on top:

In [ ]:
custom_input_tensor = tf.placeholder(dtype=tf.float32)

# this is our modified Keras model
model = Sequential()
model.add(InputLayer(input_tensor=custom_input_tensor,
                     input_shape=(None, 784)))

# build the rest of the model as before
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))

At this stage, you can call `model.load_weights(weights_file)` to load your pre-trained weights.

* Then you will probably want to collect the Sequential model's output tensor.
* You can now add new TensorFlow ops on top of output_tensor, etc.

In [ ]:
output_tensor = model.output

### Calling a Keras model on a TensorFlow tensor

* A Keras model acts the same as a layer, and thus can be called on TensorFlow tensors.
* Note: by calling a Keras model, your are reusing both its architecture and its weights.
* When you are calling a model on a tensor, you are creating new TF ops on top of the input tensor.
* These new ops are reusing the TF Variable instances already present in the model.

In [4]:
from keras.models import Sequential

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=784))
model.add(Dense(10, activation='softmax'))

# this works! 
x = tf.placeholder(tf.float32, shape=(None, 784))
y = model(x)

### Multi-GPU and distributed training

#### Assigning part of a Keras model to different GPUs
* TensorFlow device scopes are fully compatible with Keras layers and models, hence you can use them to assign specific parts of a graph to different GPUs. Here's a simple example:

In [ ]:
with tf.device('/gpu:0'):
    x0 = tf.placeholder(tf.float32, shape=(None, 20, 64))
    y0 = LSTM(32)(x)  # all ops in the LSTM layer will live on GPU:0

with tf.device('/gpu:1'):
    x1 = tf.placeholder(tf.float32, shape=(None, 20, 64))
    y1 = LSTM(32)(x)  # all ops in the LSTM layer will live on GPU:1

* Note that the variables created by the LSTM layers will not live on GPU: all TensorFlow variables always live on CPU independently from the device scope where they were created.
* TensorFlow handles device-to-device variable transfer behind the scenes.
* If you want to train multiple replicas of a same model on different GPUs, while sharing the same weights across the different replicas, you should first instantiate your model (or layers) under one device scope, then call the same model instance multiple times in different GPU device scopes, such as:

In [3]:
import os
import numpy as np
data = np.random.normal(size=7840).reshape(-1,784)
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8252968025459123186
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10909243802
locality {
  bus_id: 1
  links {
    link {
      device_id: 1
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 9404763384781821938
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:0a:00.0, compute capability: 6.1"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 8309797684
locality {
  bus_id: 1
  links {
    link {
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 7233237150910799867
physical_device_desc: "device: 1, name: GeForce GTX 1080 Ti, pci bus id: 0000:43:00.0, compute capability: 6.1"
]


In [6]:
init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()

with tf.device('/cpu:0'):
    x = tf.placeholder(tf.float32, shape=(None, 784))

    # shared model living on CPU:0
    # it won't actually be run during training; it acts as an op template
    # and as a repository for shared variables
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim=784))
    model.add(Dense(10, activation='softmax'))

# replica 0
with tf.device('/gpu:0'):
    output_0 = model(x)  # all ops in the replica will live on GPU:0

# replica 1
with tf.device('/gpu:1'):
    output_1 = model(x)  # all ops in the replica will live on GPU:1

# merge outputs on CPU
with tf.device('/cpu:0'):
    preds = 0.5 * (output_0 + output_1)

# we only run the `preds` tensor, so that only the two
# replicas on GPU get run (plus the merge op on CPU)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output_value = sess.run([preds], feed_dict={x: data})

### Distributed training
* You can trivially make use of TensorFlow distributed training by registering with Keras a TF session linked to a cluster.
* For more information about using TensorFlow in a distributed setting, see this tutorial: https://www.tensorflow.org/deploy/distributed

In [8]:
server = tf.train.Server.create_local_server()
sess = tf.Session(server.target)
from keras import backend as K
K.set_session(sess)

### Exporting a model with TensorFlow-serving
* TensorFlow Serving is a library for serving TensorFlow models in a production setting, developed by Google.
* Any Keras model can be exported with TensorFlow-serving (as long as it only has one input and one output, which is a limitation of TF-serving), whether or not it was training as part of a TensorFlow workflow. 
* In fact you could even train your Keras model with Theano then switch to the TensorFlow Keras backend and export your model.

Here's how it works.
* If your graph makes use of the Keras learning phase (different behavior at training time and test time), the very first thing to do before exporting your model is to hard-code the value of the learning phase (as 0, presumably, i.e. test mode) into your graph.
* This is done by 1) registering a constant learning phase with the Keras backend, and 2) re-building your model afterwards.

Here are these two simple steps in action:

In [18]:
from keras import backend as K

K.set_learning_phase(0)  # all new operations will be in test mode from now on

previous_model=model

# serialize the model and get its weights, for quick re-building
config = previous_model.get_config()
weights = previous_model.get_weights()

In [19]:
len(weights)

4

In [20]:
# re-build a model where the learning phase is now hard-coded to 0
from keras.models import model_from_config
new_model = model_from_config(config[0])
new_model.set_weights(weights=weights)

ValueError: You called `set_weights(weights)` on layer "dense_1" with a  weight list of length 4, but the layer was expecting 0 weights. Provided weights: [array([[-0.0514676 , -0.0694208 ,  0.03198149, .....

We can now use TensorFlow-serving to export the model, following the instructions found in the official tutorial: https://github.com/tensorflow/serving/blob/master/tensorflow_serving/g3doc/serving_basic.md

In [ ]:
from tensorflow_serving.session_bundle import exporter

export_path = ... # where to save the exported graph
export_version = ... # version number (integer)

saver = tf.train.Saver(sharded=True)
model_exporter = exporter.Exporter(saver)
signature = exporter.classification_signature(input_tensor=model.input,
                                              scores_tensor=model.output)
model_exporter.init(sess.graph.as_graph_def(),
                    default_graph_signature=signature)
model_exporter.export(export_path, tf.constant(export_version), sess)